In [1]:
import pandas as pd
import numpy as np
import datetime
import quandl
import config

# settings to observe x amount of rows in dataframe
pd.set_option('max_rows', 500)
pd.set_option('max_columns', 1000)

In [2]:
# to open file and build list of available stocks
stocks_data_file = pd.read_csv('/Users/Alfredo/Desktop/clustering-stocks/ZACKS-FC.csv')

stocks_to_analyze = stocks_data_file.ticker.unique()
print(stocks_to_analyze)

['AAPL' 'AXP' 'BA' 'CAT' 'CSCO' 'CVX' 'DD' 'DIS' 'GE' 'GS' 'HD' 'IBM'
 'INTC' 'JNJ' 'JPM' 'KO' 'MCD' 'MMM' 'MRK' 'MSFT' 'NKE' 'PFE' 'PG' 'TRV'
 'UNH' 'UTX' 'V' 'VZ' 'WMT' 'XOM']


In [3]:
# API Key
my_api_key = config.api_key


# Loop through tickers given
tickers = stocks_to_analyze

symbols = []

for sym in tickers:
    
    if sym in tickers:
        sym = sym.replace(".", "_")
    
    start = "2016-01-01"
    end = "2016-12-31"
    
    mydata = quandl.get("WIKI/{}.11".format(sym), start_date=start, end_date=end, api_key = my_api_key)
    mydata['Symbol'] = sym
    symbols.append(mydata)
    
    
# attach tickers tables
mydata = pd.concat(symbols)

# index 'Date' to column
mydata.reset_index(inplace=True)

# Change location of column 'Symbol'
symb = mydata['Symbol']
mydata.drop(labels=['Symbol'], axis=1,inplace = True)
mydata.insert(1, 'Symbol', symb)
mydata

,Date,Symbol,Adj. Close
0,2016-01-04,AAPL,101.783763
1,2016-01-05,AAPL,99.233131
2,2016-01-06,AAPL,97.291172
3,2016-01-07,AAPL,93.185040
4,2016-01-08,AAPL,93.677776
5,2016-01-11,AAPL,95.194629
6,2016-01-12,AAPL,96.576222
7,2016-01-13,AAPL,94.093220
8,2016-01-14,AAPL,96.151117
9,2016-01-15,AAPL,93.842021


In [4]:
# Group by symbol and Date
mydata = mydata.groupby(['Symbol','Date']).agg({'Adj. Close':'first'})

# Stock Daily Return
d_returns = mydata.groupby('Symbol').pct_change()
mydata['Daily_Return'] = d_returns
mydata.head(3)

# Mean Daily Return
d_mean_return = mydata.groupby('Symbol').Daily_Return.transform('mean').to_frame()
mydata['Mean_Daily_Return'] = d_mean_return

# Std Daily
d_std_return = mydata.groupby('Symbol').Daily_Return.transform('std').to_frame()
mydata['Std_Daily_Return'] = d_std_return

# Daily Sharpe Ratio
sr = d_mean_return/d_std_return
mydata['Daily_Sharpe_Ratio'] = sr

# Annualize Sharpe Ratio
asr = (252**0.5) * sr
mydata['Annualize_Sharpe_Ratio'] = asr

mydata

Adj. Close  Daily_Return  Mean_Daily_Return  \
Symbol Date                                                      
AAPL   2016-01-04  101.783763           NaN           0.000574   
       2016-01-05   99.233131     -0.025059           0.000574   
       2016-01-06   97.291172     -0.019570           0.000574   
       2016-01-07   93.185040     -0.042205           0.000574   
       2016-01-08   93.677776      0.005288           0.000574   
       2016-01-11   95.194629      0.016192           0.000574   
       2016-01-12   96.576222      0.014513           0.000574   
       2016-01-13   94.093220     -0.025710           0.000574   
       2016-01-14   96.151117      0.021871           0.000574   
       2016-01-15   93.842021     -0.024015           0.000574   
       2016-01-19   93.387931     -0.004839           0.000574   
       2016-01-20   93.513531      0.001345           0.000574   
       2016-01-21   93.040118     -0.005063           0.000574   
       2016-01-22   97.986799      0.053167           0.000574   
       2016-01-25   96.073825     -0.019523           0.000574   
       2016-01-26   96.605206      0.005531           0.000574   
       2016-01-27   90.257610     -0.065707           0.000574   
       2016-01-28   90.904929      0.007172           0.000574   
       2016-01-29   94.044912      0.034541           0.000574   
       2016-02-01   93.165717     -0.009349           0.000574   
       2016-02-02   91.281727     -0.020222           0.000574   
       2016-02-03   93.088425      0.019793           0.000574   
       2016-02-04   93.832360      0.007992           0.000574   
       2016-02-05   91.326278     -0.026708           0.000574   
       2016-02-08   92.287914      0.010530           0.000574   
       2016-02-09   92.268487     -0.000211           0.000574   
       2016-02-10   91.569115     -0.007580           0.000574   
       2016-02-11   91.015446     -0.006046           0.000574   
       2016-02-12   91.297138      0.003095           0.000574   
       2016-02-16   93.871214      0.028194           0.000574   
       2016-02-17   95.308811      0.015315           0.000574   
       2016-02-18   93.502101     -0.018956           0.000574   
       2016-02-19   93.288404     -0.002285           0.000574   
       2016-02-22   94.104338      0.008746           0.000574   
       2016-02-23   91.977082     -0.022605           0.000574   
       2016-02-24   93.346685      0.014891           0.000574   
       2016-02-25   93.987776      0.006868           0.000574   
       2016-02-26   94.133478      0.001550           0.000574   
       2016-02-29   93.919781     -0.002270           0.000574   
       2016-03-01   97.649763      0.039715           0.000574   
       2016-03-02   97.863460      0.002188           0.000574   
       2016-03-03   98.591972      0.007444           0.000574   
       2016-03-04  100.058710      0.014877           0.000574   
       2016-03-07   98.951372     -0.011067           0.000574   
       2016-03-08   98.135438     -0.008246           0.000574   
       2016-03-09   98.222859      0.000891           0.000574   
       2016-03-10   98.271427      0.000494           0.000574   
       2016-03-11   99.330198      0.010774           0.000574   
       2016-03-14   99.582749      0.002543           0.000574   
       2016-03-15  101.583729      0.020094           0.000574   
       2016-03-16  102.933904      0.013291           0.000574   
       2016-03-17  102.768775     -0.001604           0.000574   
       2016-03-18  102.885337      0.001134           0.000574   
       2016-03-21  102.875623     -0.000094           0.000574   
       2016-03-22  103.662416      0.007648           0.000574   
       2016-03-23  103.089320     -0.005528           0.000574   
       2016-03-24  102.642500     -0.004334           0.000574   
       2016-03-28  102.176252     -0.004542           0.000574   
       2016-03-29  104.594912      0.023671           0.000

In [5]:
# subtracs outstanding shares
mydata_shares = quandl.get_table('ZACKS/FC', ticker=['AAPL','AXP','BA','CAT'], per_end_date ={'gte':'2015-12-31','lte':'2017-01-30'}, 
                            qopts={'columns':['ticker','per_end_date', "comm_shares_out",
                                              "wavg_shares_out_diluted","wavg_shares_out"]})


# filters to start and end of year
mydata_shares = mydata_shares[mydata_shares.per_end_date == '2015-12-31']
mydata_shares

,ticker,per_end_date,comm_shares_out,wavg_shares_out_diluted,wavg_shares_out
None,,,,,
1,AAPL,2015-12-31,5544.487,5594.127,5558.93
6,AXP,2015-12-31,969.000,1003.000,999.00
8,AXP,2015-12-31,969.000,1003.000,999.00
13,BA,2015-12-31,666.624,695.000,686.90
15,BA,2015-12-31,666.624,695.000,686.90
20,CAT,2015-12-31,582.322,601.300,594.30
22,CAT,2015-12-31,582.322,601.300,594.30


#### Notes:

In Millions

In [6]:
print('wavg_shares_out_diluted','\n',
      'Weighted-average shares outstanding (diluted)','\n',
     '''The number of shares held by shareholders including insiders 
 but not including a company's treasury shares assuming conversion of all convertible 
 debt, securities, warrants and options, sourced from the company 10-K or 10-Q SEC reports.''', '\n')
print('wavg_shares_out', '\n',
      'Weighted-average shares outstanding (basic)','\n',
     '''The number of shares held by shareholders including
insiders but not including a company's treasury shares, 
sourced from the company 10-K or 10-Q SEC reports.''','\n')

print('comm_shares_out','\n','Common shares outstanding','\n',
      '''The number of class A shares currently held by investors.''')


wavg_shares_out_diluted 
 Weighted-average shares outstanding (diluted) 
 The number of shares held by shareholders including insiders 
 but not including a company's treasury shares assuming conversion of all convertible 
 debt, securities, warrants and options, sourced from the company 10-K or 10-Q SEC reports. 

wavg_shares_out 
 Weighted-average shares outstanding (basic) 
 The number of shares held by shareholders including
insiders but not including a company's treasury shares, 
sourced from the company 10-K or 10-Q SEC reports. 

comm_shares_out 
 Common shares outstanding 
 The number of class A shares currently held by investors.
